In [55]:
import torch, sys, glob, os, math
from pathlib import Path
import numpy as np
from multiprocessing import Pool


## Binarize Training Data

In [44]:
n = 16
cwd = os.getcwd()
prep_train_data_name = "ru_small_id.txt"


In [61]:
%cd data
!split -n l/{n} {prep_train_data_name} chunk_
%cd {cwd}

/project/cl-work2/armin-sa/resources/cc100-rus
/home/is/armin-sa/Projects/lm


In [59]:
def binarize_chunk(f_in_path):
    with open(f_in_path, "r") as f_in, open(
        "data/prep_" + Path(f_in_path).stem, "wb"
    ) as f_out:
        for line in f_in:
            arr = np.fromstring(line, dtype=np.int32, sep=" ")
            f_out.write(
                len(arr).to_bytes(2, byteorder=sys.byteorder, signed=False)
                + arr.tobytes()
            )
    return np.array(sample_locs)


In [62]:
with Pool(processes=n) as p:
    p.map(binarize_chunk, glob.glob("data/chunk_*"))


In [63]:
%cd data
!rm chunk_*
!cat prep_chunk_* > ru_small.bin
!rm prep_chunk_*
%cd {cwd}

/project/cl-work2/armin-sa/resources/cc100-rus
/home/is/armin-sa/Projects/lm


In [81]:
sample_locs =[0]
with open("data/ru_small.bin", "rb") as f_in:
    while count := int.from_bytes(f_in.read(2), byteorder=sys.byteorder, signed=False):
        arr = np.frombuffer(f_in.read(count * 4), count=count, dtype=np.int32)
        sample_locs.append(f_in.tell())

sample_locs=np.array(sample_locs[:-1])

In [82]:
with open("data/ru_small.bin", "rb") as f:
    f.seek(sample_locs[-1])
    count = int.from_bytes(f.read(2), byteorder=sys.byteorder, signed=False)
    arr = np.frombuffer(f.read(count * 4), count=count, dtype=np.int32)

In [50]:
    # def _gen():
    #     with open("data/ru_small.bin", "rb") as f_in:
    #         while count := int.from_bytes(f_in.read(2), byteorder=sys.byteorder, signed=False):
    #             arr = torch.frombuffer(f_in.read(count * 4), count=count, dtype=torch.int)
    #             yield arr

In [51]:
# a =_gen()
next(a)

tensor([    1,  3272,   218,    63, 16572,  6069,     4, 18632,   123,    21,
           83,     6,   671,     4,    98,  1637,   678,   113,    46, 28247,
            4,     7,  1402,     8,    14,  8618,   306,   101,  1006,     4,
           43,  1248,   275,  5130,     6, 10898,     4,    11,   413,     8,
           15,   211, 14318,     4,     2], dtype=torch.int32)

In [97]:
class CC100DataSet(torch.utils.data.Dataset):
    def __init__(self, file_path):
        super().__init__()
        self.file = open(file_path, "rb")
    def __len__(self):
        return len(sample_locs)
    def __getitem__(self, idx):
        self.file.seek(sample_locs[idx])
        count = int.from_bytes(self.file.read(2), byteorder=sys.byteorder, signed=False)
        arr = torch.frombuffer(self.file.read(count * 4), count=count, dtype=torch.int)
        return arr


ds = CC100DataSet("data/ru_small.bin")
print(list(torch.utils.data.DataLoader(ds, num_workers=4, shuffle=False))[:])

[tensor([[  1,   4,  68,   4, 101,  40,   4, 127,  24,  18,   4,  82,  69,  69,
          83,   4,  45,  20,  16,   4,   6,  13,  31,  23,   4,  40,  32,  55,
          16,   4,  12,  35,  10,  30,   4,   6,  27,  55,   9,   5,  18,  29,
          10,  19,  49,   4,  19,   8,  18,  18,   5,  19,   9,   6,  14,   5,
           4,  95,   7,  43,  24,  30,   4,  40,  30,  42,  13,  17,   4, 127,
          24,  34,  43,   7,  18,  31,  29,  19,   6,  13,  35,  12,  22,  59,
          13,   4,   6,  27,  55,   9,   5,  18,  29,  10,   9,  14,  49,   4,
         105,  26,   7,  10,   8,  62, 107,  42,   8, 127,  11,   7,  33,  31,
          10,  19,  52,   4,  64,   7,  38,   7,  13,  10,  19,  52,  43,   8,
          15,   7,  10,   9,  24,  11,  17,  61,  78,   4,  11,   8,  20,   6,
          38,  10,  29,   4,   6,  53,  17,   4,   6,  27,  55,  25,   4,  63,
          12,  15,  55,  13,  31,   9,   7,  18,  29,  59,  41,   4,   9,  11,
          12,  20,  43,   6,   9,  11,   8,  26,   